BrainTumor Project

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#define image size and batch size
IMG_SIZE=224
BATCH_SIZE=32

In [ ]:
#Define data generators for train,validation and test sets
train_datagen=ImageDataGenerator(
     rescale=1./255,
     validation_split=0.2
 )
train_generator=train_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator=train_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_datagen=ImageDataGenerator(rescale=1./255)
test_generator=test_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/test',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 60 images belonging to 1 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#define the model
model=keras.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(1,activation='sigmoid')
])

In [ ]:
#compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
75/75 [==============================] - 1004s 13s/step - loss: 0.5024 - accuracy: 0.8008 - val_loss: 0.2674 - val_accuracy: 0.9000
Epoch 2/5
75/75 [==============================] - 310s 4s/step - loss: 0.2025 - accuracy: 0.9267 - val_loss: 0.1919 - val_accuracy: 0.9400
Epoch 3/5
75/75 [==============================] - 335s 4s/step - loss: 0.0963 - accuracy: 0.9708 - val_loss: 0.0438 - val_accuracy: 0.9867
Epoch 4/5
75/75 [==============================] - 283s 4s/step - loss: 0.0446 - accuracy: 0.9862 - val_loss: 0.0410 - val_accuracy: 0.9867
Epoch 5/5
75/75 [==============================] - 283s 4s/step - loss: 0.0353 - accuracy: 0.9896 - val_loss: 0.0330 - val_accuracy: 0.9850


In [ ]:
model.save("Model.h5","label.txt")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

#Load the saved model
model=load_model('/content/Model.h5')

#Load and preprocess the test image
test_image_path='/content/drive/MyDrive/Brain_Tumor_Detection/test/pred/pred13.jpg'
img=image.load_img(test_image_path,target_size=(224,224))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array,axis=0)  #Add batch dimension
img_array /=225. #normalize the pixel values
#make predictions
prediction=model.predict(img_array)
#print the prediction
if prediction <0.5:
   print("prediction: No tumor (probability:",prediction[0][0],")")
else:
   print("prediction:tumor present (probability:",prediction[0][0],")")

1/1 [==============================] - 0s 185ms/step
prediction:tumor present (probability: 0.8242763 )
